In [1]:
import pandas as pd
from pathlib import Path
import pymc as pm
import plotly.express as px
import numpy as np
import pandas as pd

from estival.model import BayesianCompartmentalModel
import estival.priors as esp
import estival.targets as est
from estival.wrappers import pymc as epm
from tbdynamics import model
from tbdynamics.utils import round_sigfig
from tbdynamics.inputs import fixed_parameters
import plotly.graph_objects as go
from tbdynamics.constants import age_strata, organ_strata, compartments, latent_compartments, infectious_compartments

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


## Define variables

In [2]:
PROJECT_PATH = Path().resolve()
DATA_PATH = PROJECT_PATH / 'data'
pd.options.plotting.backend = "plotly"

time_start = 1700.0
time_end = 2023.0
time_step = 0.1
matrix = np.ones((6, 6)) # Homo mixing


## Define Model

### Base model

In [3]:
tb_model = model.build_model(
    compartments,
    latent_compartments,
    infectious_compartments,
    age_strata,
    time_start,
    time_end,
    time_step,
    fixed_parameters,
    matrix
)

### Params and calibration targets

In [4]:
params = {}


priors = [
    esp.UniformPrior("start_population_size", (100000, 30000000)),
    esp.UniformPrior("contact_rate", (0, 10)),  # multiplied with eigenvalue
    esp.UniformPrior("rr_infection_latent", (0.2, 0.5)),
    esp.UniformPrior("rr_infection_recovered", (0.1, 0.5)),
    esp.UniformPrior("progression_multiplier", (1.0, 5.0)),
    esp.UniformPrior("seed_time", [1700.0, 1900.0]),
    esp.UniformPrior("seed_duration", [1.0, 5.0]),
    esp.UniformPrior("seed_num", [1.0, 1000.0]),
    esp.UniformPrior("smear_positive_death_rate", (0.335, 0.449)),
    esp.UniformPrior("smear_negative_death_rate", (0.017, 0.035)),
    esp.UniformPrior("smear_positive_self_recovery", (0.177, 0.288)),
    esp.UniformPrior("smear_negative_self_recovery", (0.073, 0.209)),
]

pop = pd.Series(
    {
        2009: 86025000,
        2019: 96484000,
    }
)

notifs = pd.Series(
    {
        2010: 99022,
        2011: 100518,
        2012: 103906,
        2013: 102196,
        2014: 102087,
        2015: 102676,
        2016: 102527,
        2017: 105733,
        2018: 102171,
        2019: 104505,
        2020: 101795,
        2021: 78935,
        2022: 103804,
    }
)

targets = [
    est.NormalTarget("total_population", pop, stdev=100000),
    est.NormalTarget("notification", notifs, stdev=1000),
]

calibration_model = BayesianCompartmentalModel(tb_model, params, priors, targets)

### Running Calibration

In [5]:
with pm.Model() as pmc_model:
    start_params = {k: np.clip(v, *calibration_model.priors[k].bounds(0.99)) for k, v in params.items() if k in calibration_model.priors}
    variables = epm.use_model(calibration_model)
    map_params = pm.find_MAP(start=start_params, vars=variables, maxeval= 10000,include_transformed=False)
    map_params = {k: float(v) for k, v in map_params.items()}
    print('Best calibration parameters found:')
for i_param, param in enumerate(map_params):
    print(f'   {param}: {round_sigfig(map_params[param], 4)} (within bound {priors[i_param].bounds()}')


Best calibration parameters found:
   start_population_size: 337900.0 (within bound (100000.0, 30000000.0)
   contact_rate: 3.781 (within bound (0.0, 10.0)
   rr_infection_latent: 0.2 (within bound (0.2, 0.5)
   rr_infection_recovered: 0.1 (within bound (0.1, 0.5)
   progression_multiplier: 2.595 (within bound (1.0, 5.0)
   seed_time: 1721.0 (within bound (1700.0, 1900.0)
   seed_duration: 4.453 (within bound (1.0, 5.0)
   seed_num: 442.6 (within bound (1.0, 1000.0)
   smear_positive_death_rate: 0.335 (within bound (0.335, 0.449)
   smear_negative_death_rate: 0.035 (within bound (0.017, 0.035)
   smear_positive_self_recovery: 0.2872 (within bound (0.177, 0.288)
   smear_negative_self_recovery: 0.07473 (within bound (0.073, 0.20900000000000002)
   acf_inflection_time: 2020.0 (within bound (2000.0, 2020.0)
   acf_start_asymp: 0.1 (within bound (0.07, 0.1)
   acf_end_asymp: 0.4 (within bound (0.4, 0.55)


In [6]:
params.update(map_params)
tb_model.run(params)
derived_df_0 = tb_model.get_derived_outputs_df()

### Population output

In [7]:
line_trace = go.Scatter(
    x=derived_df_0.index,
    y=derived_df_0["total_population"],
    mode="lines",
    name="Modeled Data",
)
# Create a scatter plot for the actual data
scatter_trace = go.Scatter(
    x=pop.index,
    y=pop.values,
    mode="markers",
    marker=dict(color="red"),
    name="Actual Data",
)

# Combine the traces into one figure
fig = go.Figure(data=[line_trace, scatter_trace])

# Update the layout for the combined figure
fig.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Population"
)

# Show the figure
fig.show()

In [8]:
derived_df_0[[f'total_populationXage_{i}' for i in age_strata]].plot(title='Modelled populatation by age group', kind ='area')

In [9]:
derived_df_0['prevalence_infectious'].plot()

In [10]:
derived_df_0[[f"prop_{compartment}" for compartment in compartments]].plot(kind="area")

In [11]:
derived_df_0[[f'total_populationXorgan_{i}' for i in organ_strata]].plot(title='Modelled populatation by organ status', kind ='area')

In [12]:
derived_df_0[[f"prop_{organ_stratum}" for organ_stratum in organ_strata]].plot(kind="area")

In [13]:
line_trace = go.Scatter(
    x=derived_df_0.index,
    y=derived_df_0["notification"],
    mode="lines",
    name="Modeled Data",
)
# Create a scatter plot for the actual data
scatter_trace = go.Scatter(
    x=notifs.index,
    y=notifs.values,
    mode="markers",
    marker=dict(color="red"),
    name="Actual Data",
)

# Combine the traces into one figure
fig = go.Figure(data=[line_trace, scatter_trace])

# Update the layout for the combined figure
fig.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Notifications"
)

# Show the figure
fig.show()